## Feature extraction with data augmentation

#### Freezing the convolutional base in a neural network means preventing its weights from being updated during training. This is crucial because if the pre-trained representations in the convolutional base were modified, the randomly initialized Dense layers on top would propagate very large weight updates, effectively destroying the valuable learned representations. In Keras, this is achieved by setting the trainable attribute of the layer or model to False.

In [1]:
import keras
import numpy as np
from keras import layers
from keras.utils import image_dataset_from_directory
import os, shutil, pathlib

In [2]:
original_dir = pathlib.Path("cats-dogs-images")
new_base_dir = pathlib.Path("cats_vs_dogs_subset")

train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(180, 180),
    batch_size=32
)
validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size=(180, 180),
    batch_size=32
)
test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(180, 180),
    batch_size=32
)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


In [11]:
conv_base = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False # meaning we won't be using the pretrained model's Dense classifier layers as it contains 1000 classifiers we won't need, we will be adding our own dense layer classifiers as we only need 2 classifier.
)

In [10]:
conv_base.trainable = True
print("This is the number of trainable weights " 
"before freezing the conv base:", len(conv_base.trainable_weights))

conv_base.trainable = False
print("This is the number of trainable weights " 
"after freezing the conv base:", len(conv_base.trainable_weights))

This is the number of trainable weights before freezing the conv base: 26
This is the number of trainable weights after freezing the conv base: 0


<h5>We require 3 things:</h5>
<ul style="list-style-type: disc; color: blue; font-family: Arial, sans-serif;">
    <li>Data augmentation stage</li>
    <li>Frozen convolution base [conv_base] from above</li>
    <li>A Dense classifier</li>
</ul>

In [14]:
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0)
])

inputs = keras.Input(shape=(180,180,3))
x = data_augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(
    loss="binary_crossentropy",
    optimizer="rmsprop",
    metrics=["accuracy"]
)

##### When configuring a neural network in this manner, only the weights of the two added Dense layers will be updated during training. This amounts to four weight tensors in total: a primary weight matrix and a bias vector for each of the two layers. It's crucial to compile the model after setting up this configuration for the changes to take effect. If you modify the trainability of weights after initial compilation, you must recompile the model; otherwise, your adjustments will be disregarded.  This technique is expensive enough that you should only attempt it if you have access to a GPU (such as the free GPU available in Colab)—it’s intractable on CPU. If you can’t run your code on GPU, then the previous technique is the way to go.

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
    filepath="feature-extraction-with-pretrained-model-data-augmentation.keras",
    save_best_only=True,
    monitor="val_loss")
 ]
history = model.fit(
    train_dataset,
    epochs=5,
    validation_data=validation_dataset,
    callbacks=callbacks
)